# Base Distillation

the baseline for the distillation matrics, including:
- a converged teacher ResNet-56 on CIFAR-100
- a converged student ResNet-20 on CIFAR-100
- a converged student ResNet-20 with Knowledge Distillation on CIFAR-100

the accuracy of these three model will served as the benchmark for the rest experient

In [2]:
from utils.download_cifar100 import download_cifar100

# 先加载数据集 CIFAR-100
trainset, testset, trainloader, testloader = download_cifar100()

开始下载CIFAR-100数据集...
下载训练集中...
下载测试集中...
下载完成！
训练集样本数: 50000
测试集样本数: 10000
类别数: 100


In [4]:
# ResNet
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    """ResNet基础块"""
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x, return_feature=False):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    """ResNet模型，支持特征返回"""
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_feature=False):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        feature = out.view(out.size(0), -1)
        out = self.linear(feature)

        if return_feature:
            return out, feature
        return out

def ResNet20(num_classes=100):
    return ResNet(BasicBlock, [3, 3, 3, 3], num_classes=num_classes)

def ResNet56(num_classes=100):
    return ResNet(BasicBlock, [9, 9, 9, 9], num_classes=num_classes)

In [5]:
def test_model(model, testloader, device):
    """测试模型准确率"""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100. * correct / total

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import time


"""训练教师模型 ResNet-56"""
print("🚀 开始训练教师模型 ResNet-56...")

# 模型和优化器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = ResNet56(num_classes=100).to(device)

optimizer = optim.SGD(teacher_model.parameters(), lr=0.1,
                     momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                          milestones=[100, 150], gamma=0.1)
criterion = nn.CrossEntropyLoss()

# 训练记录
train_losses = []
test_accuracies = []
best_acc = 0

# 训练循环
for epoch in range(200):
    teacher_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(trainloader, desc=f'Epoch {epoch+1}/200')
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = teacher_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        pbar.set_postfix({
            'Loss': f'{loss.item():.3f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })

    # 学习率调度
    scheduler.step()

    # 测试准确率
    test_acc = test_model(teacher_model, testloader, device)
    test_accuracies.append(test_acc)
    train_losses.append(running_loss/len(trainloader))

    print(f'Epoch {epoch+1}: Test Accuracy = {test_acc:.2f}%')

    # 保存最佳模型
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(teacher_model.state_dict(), 'teacher_resnet56_best.pth')
        print(f'✅ 新的最佳准确率: {best_acc:.2f}%')

# 保存最终模型
torch.save(teacher_model.state_dict(), 'teacher_resnet56_final.pth')
print(f'🎉 教师模型训练完成! 最佳准确率: {best_acc:.2f}%')


🚀 开始训练教师模型 ResNet-56...


Epoch 1/200:   1%|          | 4/391 [00:06<05:46,  1.12it/s, Loss=7.536, Acc=1.37%]